#### Rangkuman hasil proyek 
1. Metode representasi teks countvectorizer menghasilkan model yang akurat meski dengan jumlah dataset yang kecil.
2. Metode representasi teks word2vec kurang efektif dalam menghasilkan model yang akurat karena memerlukan jumlah dataset yang besar untuk dapat menangkap hubungan semantik antarkata dengan baik.
3. Model Random Forest Classifier dapat menjadi lebih akurat daripada SVC disebabkan RFC lebih tahan terhadap overfitting dibandingkan model Support Vector Classifier.

# Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from nlp_id.tokenizer import Tokenizer 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nlp_id.lemmatizer import Lemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

warnings.filterwarnings('ignore')

# Data gathering (Webscraping)

- Untuk memenuhi task ini anda perlu mengumpulkan sebanyak minimal 10 data teks berita yang berasal dari media digital seperti liputan 6, detik.com dan lain-lain dengan kategori berita politik atau olahraga atau hiburan. Data ini harus berasal minimal dari 3 media yang berbeda dan anda melakukan pelabelan secara manual.

In [2]:
def webscrape(url, label, website_name, class_name):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        div_element = soup.find("div", class_=class_name)
        news_paragraphs = div_element.find_all("p")
        combined_text = ""
        for paragraph in news_paragraphs:
            paragraph_text = paragraph.get_text().strip()    
            combined_text += paragraph_text + " "
        return {
            "text": combined_text,
            "media": website_name,
            "label": label
        }
    except Exception as e:
        return {"error": str(e)}

# List of dictionaries containing URLs, labels, website names, and class names
data_to_scrape = [
    {"url": "https://www.liputan6.com/bola/read/5444476/manchester-united-lagi-terpuruk-banget-pemainnya-malah-berulah?page=2", "label": "olahraga", "website_name": "liputan6", "class_name": "article-content-body__item-page"},
    {"url": "https://www.liputan6.com/bola/read/5440093/bola-ganjil-para-peneror-mantan-robert-lewandowski-jebol-gawang-eks-klub-27-kali-di-26-pertandingan?page=3", "label": "olahraga", "website_name": "liputan6", "class_name": "article-content-body__item-page"},
    {"url": "https://sport.detik.com/sepakbola/bola-dunia/d-7014721/suarez-ke-messi-ayo-kejar-ballon-dor-kesembilan", "label": "olahraga", "website_name": "detik.com", "class_name": "detail__body-text"},
    {"url": "https://sport.detik.com/sepakbola/liga-spanyol/d-7021125/madrid-terlalu-bergantung-pada-gol-gol-bellingham", "label": "olahraga", "website_name": "detik.com", "class_name": "detail__body-text"},
    {"url": "https://mediaindonesia.com/sepak-bola/627052/anwar-el-ghazi-tidak-gentar-bela-palestina-meski-dipecat-oleh-klub-yang-dibela", "label": "olahraga", "website_name": "media indonesia", "class_name": "jap"},
    {"url": "https://www.liputan6.com/showbiz/read/5389218/5-rekomendasi-film-warkop-dki-di-vidio-hiburan-sambil-nostalgia?page=2", "label": "hiburan", "website_name": "liputan6", "class_name": "article-content-body__item-page"},
    {"url": "https://www.liputan6.com/on-off/read/5388638/film-masti-zaade-di-vidio-hiburan-bollywood-yang-tidak-boleh-anda-lewatkan?page=2", "label": "hiburan", "website_name": "liputan6", "class_name": "article-content-body__item-page"},
    {"url": "https://hot.detik.com/celeb/d-7019026/profil-kia-poetri-cewek-yang-dikabarkan-dekat-dengan-virgoun", "label": "hiburan", "website_name": "detik.com", "class_name": "detail__body-text"},
    {"url": "https://hot.detik.com/berita-kpop/d-7025764/terungkap-tanggal-pernikahan-sojin-girls-day-dan-lee-dong-ha", "label": "hiburan", "website_name": "detik.com", "class_name": "detail__body-text"},
    {"url": "https://wolipop.detik.com/entertainment-news/d-7018236/jejak-lisa-blackpink-dihapus-celine-dan-bvlgari-imbas-kontroversi-di-paris", "label": "hiburan", "website_name": "detik.com", "class_name": "itp_bodycontent"},
    {"url": "https://mediaindonesia.com/hiburan/623956/jisoo-blackpink-dan-aktor-ahn-bo-hyun-putus-ada-apa", "label": "hiburan", "website_name": "media indonesia", "class_name": "jap"},
    {"url": "https://www.liputan6.com/regional/read/5439797/tidak-terpengaruh-lawan-politik-prabowo-diprediksi-tetap-jadi-pilihan-utama-warga-jabar-di-pilpres-2024", "label": "politik", "website_name": "liputan6", "class_name": "article-content-body__item-page"},
    {"url": "https://www.liputan6.com/pemilu/read/5438026/syahganda-nainggolan-anies-baswedan-jenius-dan-memiliki-kesabaran-politik-yang-tinggi", "label": "politik", "website_name": "liputan6", "class_name": "article-content-body__item-page"},
    {"url": "https://www.detik.com/jatim/berita/d-7019259/puan-ungkap-pesan-khusus-megawati-ke-dirinya-untuk-pilpres-2024", "label": "politik", "website_name": "detik.com", "class_name": "detail__body-text"},
    {"url": "https://www.detik.com/sumut/berita/d-7019256/ikuti-relawan-bobby-nasution-tegaskan-dukung-prabowo-gibran", "label": "politik", "website_name": "detik.com", "class_name": "detail__body-text"},
    {"url": "https://mediaindonesia.com/politik-dan-hukum/627084/mkmk-harus-dalami-kejahatan-terencana-dan-terorganisir-terkait-putusan-gibran", "label": "politik", "website_name": "media indonesia", "class_name": "jap"}
]

# List to store scraped data
scraped_data = []

# Scrape data for each URL, label, website name, and class name and append it to the scraped_data list
for item in data_to_scrape:
    result = webscrape(item["url"], item["label"], item["website_name"], item["class_name"])
    # Append the result to the scraped_data list
    scraped_data.append(result)

# Print the scraped data
df = pd.DataFrame(scraped_data)

Saya mengumpulkan urls dari 15 website perpaduan dari website liputan6, detik.com, dan media indonesia, dimana setiap website memiliki keunikan class di bagian html website nya masing-masing, oleh karena itu, kode webscrape harus dimodifikasi secara manual, begitu juga dengan bagian labellingnya.

In [3]:
# Print out as csv for submission
#df.to_csv('C:/Users/user/downloads/2501975684_Data.csv')

In [4]:
scraped_data

[{'text': 'Liputan6.com, Jakarta- Manchester United menghadapi masa suram di awal musim 2023/2024. Mereka sudah kalah delapan kali dari 15 pertandingan awal. Di saat sedang terpuruk, malah ada satu pemain pilar MU yang berulah sehingga membuat kesal pelatih Erik ten Hag. MU dua kali kalah telak 0-3 di kandang sendiri berturut-turut. Yang pertama mereka dipermak rival sekota Manchester City di Old Trafford pada ajang Liga Inggris akhir pekan kemarin. Kemudian MU dihajar dengan skor sama di tempat yang sama oleh Newcastle United di Piala Liga atau Carabao Cup. Baca Juga Delapan kekalahan dari 15 laga awal musim ini menjadi start terburuk MU sejak 1962/1963. Ten Hag pun pusing tujuh keliling. Pria Belanda itu mulai terancam pemecatan dari MU. Di saat tim sedang babak belur, penyerang MU Marcus Rashford malah kedapatan merayakan ulang tahun dengan pesta pora. Perayaan dilakukan Rashford hanya selang beberapa jam usai MU dihajar City. Fans pun murka. Erik Ten Hag juga kesal dengan ulah Rash

Dilihat dari data-datanya, terlihat banyak huruf kapital, kata-kata yang kurang membantu dalam tahap modelling seperti liputan6, detik.com, advertisement content, dan penomoran. Maka, di tahap selanjutnya, semua ini akan di clean menggunakan teknik substitusi dan replace.

# Data Cleaning

- Tujuan proses awal ini adalah menghasilkan suatu
set vektor dari data teks yang sudah dibersihkan dari simbol, hashtag, kata-kata “https”,
ataupun “.com”, dimana keseluruhan teks harus lower case dengan size vektor adalah 50, dan
frequency kata yang diperhitungkan minimal 3

In [5]:
# Remove symbols, hashtags, https, .coms, numbers, multiple whitespaces
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'https\S+|www\S+|https|.com', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove symbols and punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip() # Remove whitespaces
    return text

# Apply preprocess to text
df['text'] = df['text'].apply(preprocess_text)

df['text'][2]

'luis suarez mengomentari kesuksesan lionel messi meraih ballon dor suarez berkelakar meminta messi mengejar trofi ballon dor yang kesembilan messi terpilih sebagai pemenang ballon dor dalam seremoni di theatre du chatalet paris selasa dini hari wib bintang asal argentina itu mengungguli erling haaland dan kylian mbappe kemenangan ini menajamkan rekor messi di ballon dor mantan pemain barcelona itu kini sudah memenangi trofi ini sebanyak delapan kali advertisement scroll to continue with content luis suarez menyebut mantan rekannya di barcelona itu pantas meraih ballon dor luar biasa sekarang kerja keraslah dan kejar yang kesembilan bersenangsenanglah kamu pantas mendapatkannya suarez berkomentar di unggahan messi di media sosial kendati demikian lionel messi memberi indikasi bahwa ini akan jadi ballon dor terakhirnya kenyataannya adalah saya sudah lama berhenti memikirkan ballon dor saya pernah bilang kalau itu tidak pernah jadi prioritas saya dan sekarang berkurang setelah meraih seg

In [6]:
# Replace noise with an empty string in the 'text' column
df['text'] = df['text'].str.replace('liputan', '')
df['text'] = df['text'].str.replace('detik', '')
df['text'] = df['text'].str.replace('media indonesia', '')
df['text'] = df['text'].str.replace('offcial', 'official')
df['text'] = df['text'].str.replace('advertisement scroll to continue with content', '')
df['text'] = df['text'].str.replace('baca juga', '')
df['text'][4]

'pesepak bola profesional anwar el ghazi konsisten mendukung kemerdekaan palestina meskipun harus dikeluarkan dari klub yang dia bela saat ini yaitu mainz yang merupakan klub bundesliga atau liga utama jerman pemain timnas belanda yang memiliki darah keturunan maroko ini tidak menyesali tindakannya menurutnya membela palestina merupakan hal yang benar untuk dilakukan berdiri untuk apa yang saya anggap benar tidak masalah bahkan jika itu berarti saya berdiri sendiri hilangnya mata pencaharian saya tidak berarti apaapa jika dibandingkan dengan apa yang dilepaskan pada orang yang tidak bersalah dan rentan di gaza hentikan pembunuhan ungkapnya dalam keterangan resmi dilansir dari mirror  erdogan upayakan gencatan senjata di gaza hentikan kejahatan kemanusiaan pria yang baru berusia tahun tersebut diketahui telah malang melintang membela beberapa klub terkenal di antaranya ajax lille aston villa everton psv dan terakhir ialah mainz mainz menganggap el ghazi telah mengambil sikap yang tak bi

Data sudah bersih dari url, simbol, tanda baca, penomoran, multiple whitespaces, dan kata-kata irelevan. Selanjutnya, lanjut ke tahap tokenizing agar teks dapat di proses oleh model.

# Word Tokenization

In [7]:
tokenizer = Tokenizer()

# Define a function for tokenization
def tokenize_text(text):
    return tokenizer.tokenize(text)

# Apply the tokenization function to the 'Teks' column using apply
df['Tokenized_Text'] = df['text'].apply(tokenize_text)

# Print the DataFrame with tokenized words
df

,text,media,label,Tokenized_Text
0,jakarta manchester united menghadapi masa sur...,liputan6,olahraga,"[jakarta, manchester, united, menghadapi, masa..."
1,jakarta mereka pernah menjadi pahlawan dan di...,liputan6,olahraga,"[jakarta, mereka, pernah, menjadi, pahlawan, d..."
2,luis suarez mengomentari kesuksesan lionel mes...,detik.com,olahraga,"[luis, suarez, mengomentari, kesuksesan, lione..."
3,real madrid harus puas dengan hasil usai dired...,detik.com,olahraga,"[real, madrid, harus, puas, dengan, hasil, usa..."
4,pesepak bola profesional anwar el ghazi konsis...,media indonesia,olahraga,"[pesepak, bola, profesional, anwar, el, ghazi,..."
5,jakarta grup lawak legendaris indonesia warko...,liputan6,hiburan,"[jakarta, grup, lawak, legendaris, indonesia, ..."
6,jakarta salah satu film bollywood bergenre ko...,liputan6,hiburan,"[jakarta, salah, satu, film, bollywood, bergen..."
7,artis kia poetri tengah menjadi sorotan publik...,detik.com,hiburan,"[artis, kia, poetri, tengah, menjadi, sorotan,..."
8,oktober lalu rencana pernikahan sojin girls da...,detik.com,hiburan,"[oktober, lalu, rencana, pernikahan, sojin, gi..."
9,penampilan lisa blackpink di kelab kabaret cra...,detik.com,hiburan,"[penampilan, lisa, blackpink, di, kelab, kabar..."


Data sudah di tokenize menggunakan library nlp-id. Selanjutnya, untuk meningkatkan kualitas data, kita melakukan proses stemming dan lematisasi

# Stemming & Lemmatization

In [8]:
# Initialize Sastrawi Stemmer and Lemmatizer
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

# Function to perform stemming on a list of words using Sastrawi
def stem_words(words):
    return [stemmer.stem(word) for word in words]

# Apply stemming and lemmatization functions to the 'Tokenized_Text' column using apply
df['Stemmed_Text'] = df['Tokenized_Text'].apply(stem_words)

# Print the DataFrame with stemmed words
df

,text,media,label,Tokenized_Text,Stemmed_Text
0,jakarta manchester united menghadapi masa sur...,liputan6,olahraga,"[jakarta, manchester, united, menghadapi, masa...","[jakarta, manchester, united, hadap, masa, sur..."
1,jakarta mereka pernah menjadi pahlawan dan di...,liputan6,olahraga,"[jakarta, mereka, pernah, menjadi, pahlawan, d...","[jakarta, mereka, pernah, jadi, pahlawan, dan,..."
2,luis suarez mengomentari kesuksesan lionel mes...,detik.com,olahraga,"[luis, suarez, mengomentari, kesuksesan, lione...","[luis, suarez, komentar, sukses, lionel, messi..."
3,real madrid harus puas dengan hasil usai dired...,detik.com,olahraga,"[real, madrid, harus, puas, dengan, hasil, usa...","[real, madrid, harus, puas, dengan, hasil, usa..."
4,pesepak bola profesional anwar el ghazi konsis...,media indonesia,olahraga,"[pesepak, bola, profesional, anwar, el, ghazi,...","[sepak, bola, profesional, anwar, el, ghazi, k..."
5,jakarta grup lawak legendaris indonesia warko...,liputan6,hiburan,"[jakarta, grup, lawak, legendaris, indonesia, ...","[jakarta, grup, lawak, legendaris, indonesia, ..."
6,jakarta salah satu film bollywood bergenre ko...,liputan6,hiburan,"[jakarta, salah, satu, film, bollywood, bergen...","[jakarta, salah, satu, film, bollywood, genre,..."
7,artis kia poetri tengah menjadi sorotan publik...,detik.com,hiburan,"[artis, kia, poetri, tengah, menjadi, sorotan,...","[artis, kia, poetri, tengah, jadi, sorot, publ..."
8,oktober lalu rencana pernikahan sojin girls da...,detik.com,hiburan,"[oktober, lalu, rencana, pernikahan, sojin, gi...","[oktober, lalu, rencana, nikah, sojin, girls, ..."
9,penampilan lisa blackpink di kelab kabaret cra...,detik.com,hiburan,"[penampilan, lisa, blackpink, di, kelab, kabar...","[tampil, lisa, blackpink, di, kelab, kabaret, ..."


In [9]:
lemmatizer = Lemmatizer() 

# Function to perform lemmatization on a list of words
def lemmatize_words(words):
    return [lemmatizer.lemmatize(word) for word in words]

# Apply stemming and lemmatization functions to the 'Tokenized_Teks' column using apply
df['Lemmatized_Text'] = df['Stemmed_Text'].apply(lemmatize_words)

# Print the DataFrame with stemmed and lemmatized words
df

,text,media,label,Tokenized_Text,Stemmed_Text,Lemmatized_Text
0,jakarta manchester united menghadapi masa sur...,liputan6,olahraga,"[jakarta, manchester, united, menghadapi, masa...","[jakarta, manchester, united, hadap, masa, sur...","[jakarta, manchester, united, hadap, masa, sur..."
1,jakarta mereka pernah menjadi pahlawan dan di...,liputan6,olahraga,"[jakarta, mereka, pernah, menjadi, pahlawan, d...","[jakarta, mereka, pernah, jadi, pahlawan, dan,...","[jakarta, mereka, pernah, jadi, pahlawan, dan,..."
2,luis suarez mengomentari kesuksesan lionel mes...,detik.com,olahraga,"[luis, suarez, mengomentari, kesuksesan, lione...","[luis, suarez, komentar, sukses, lionel, messi...","[luis, suarez, komentar, sukses, lionel, messi..."
3,real madrid harus puas dengan hasil usai dired...,detik.com,olahraga,"[real, madrid, harus, puas, dengan, hasil, usa...","[real, madrid, harus, puas, dengan, hasil, usa...","[real, madrid, harus, puas, dengan, hasil, usa..."
4,pesepak bola profesional anwar el ghazi konsis...,media indonesia,olahraga,"[pesepak, bola, profesional, anwar, el, ghazi,...","[sepak, bola, profesional, anwar, el, ghazi, k...","[sepak, bola, profesional, anwar, el, ghazi, k..."
5,jakarta grup lawak legendaris indonesia warko...,liputan6,hiburan,"[jakarta, grup, lawak, legendaris, indonesia, ...","[jakarta, grup, lawak, legendaris, indonesia, ...","[jakarta, grup, lawak, legendaris, indonesia, ..."
6,jakarta salah satu film bollywood bergenre ko...,liputan6,hiburan,"[jakarta, salah, satu, film, bollywood, bergen...","[jakarta, salah, satu, film, bollywood, genre,...","[jakarta, salah, satu, film, bollywood, genre,..."
7,artis kia poetri tengah menjadi sorotan publik...,detik.com,hiburan,"[artis, kia, poetri, tengah, menjadi, sorotan,...","[artis, kia, poetri, tengah, jadi, sorot, publ...","[artis, kia, poetri, tengah, jadi, sorot, publ..."
8,oktober lalu rencana pernikahan sojin girls da...,detik.com,hiburan,"[oktober, lalu, rencana, pernikahan, sojin, gi...","[oktober, lalu, rencana, nikah, sojin, girls, ...","[oktober, lalu, rencana, nikah, sojin, girls, ..."
9,penampilan lisa blackpink di kelab kabaret cra...,detik.com,hiburan,"[penampilan, lisa, blackpink, di, kelab, kabar...","[tampil, lisa, blackpink, di, kelab, kabaret, ...","[tampil, lisa, blackpink, di, kelab, kabaret, ..."


Data teks kita sudah di potong dan di proses untuk hanya  memiliki kata dasar makna dari sebuah kata menggunakan library Sastrawi dan nlp-id. Sebelum dilakukan representasi teks, kita pertama akan menghilangkan kata kata yang kurang memiliki makna yang berarti bagi model atau stopwords agar kualitas data semakin baik dan model akan menjadi lebih akurat dalam memprediksi.

# Removing Stopwords 

- Vektor yang dihasilkan harus berdasarkan hasil
training dengan input kata dan target output conteks dan tanpa mengandung stopwords.


In [10]:
# Remove stopwords
stop_words = set(stopwords.words('indonesian'))

# Function to remove stopwords from a text and return a list of words
def remove_stopwords(text):
    filtered_words = [word for word in text if word.lower() not in stop_words]  # Remove stopwords
    return filtered_words  # Return a list of words without stopwords

# Apply the remove_stopwords function to the 'Lemmatized_Text' column
df['Lemmatized_Text'] = df['Lemmatized_Text'].apply(remove_stopwords)

# Print the DataFrame with Lemmatized_Text column containing lists of words without stopwords
df

,text,media,label,Tokenized_Text,Stemmed_Text,Lemmatized_Text
0,jakarta manchester united menghadapi masa sur...,liputan6,olahraga,"[jakarta, manchester, united, menghadapi, masa...","[jakarta, manchester, united, hadap, masa, sur...","[jakarta, manchester, united, hadap, suram, mu..."
1,jakarta mereka pernah menjadi pahlawan dan di...,liputan6,olahraga,"[jakarta, mereka, pernah, menjadi, pahlawan, d...","[jakarta, mereka, pernah, jadi, pahlawan, dan,...","[jakarta, pahlawan, dieluelukan, dukung, klub,..."
2,luis suarez mengomentari kesuksesan lionel mes...,detik.com,olahraga,"[luis, suarez, mengomentari, kesuksesan, lione...","[luis, suarez, komentar, sukses, lionel, messi...","[luis, suarez, komentar, sukses, lionel, messi..."
3,real madrid harus puas dengan hasil usai dired...,detik.com,olahraga,"[real, madrid, harus, puas, dengan, hasil, usa...","[real, madrid, harus, puas, dengan, hasil, usa...","[real, madrid, puas, hasil, redam, rayo, valle..."
4,pesepak bola profesional anwar el ghazi konsis...,media indonesia,olahraga,"[pesepak, bola, profesional, anwar, el, ghazi,...","[sepak, bola, profesional, anwar, el, ghazi, k...","[sepak, bola, profesional, anwar, el, ghazi, k..."
5,jakarta grup lawak legendaris indonesia warko...,liputan6,hiburan,"[jakarta, grup, lawak, legendaris, indonesia, ...","[jakarta, grup, lawak, legendaris, indonesia, ...","[jakarta, grup, lawak, legendaris, indonesia, ..."
6,jakarta salah satu film bollywood bergenre ko...,liputan6,hiburan,"[jakarta, salah, satu, film, bollywood, bergen...","[jakarta, salah, satu, film, bollywood, genre,...","[jakarta, salah, film, bollywood, genre, komed..."
7,artis kia poetri tengah menjadi sorotan publik...,detik.com,hiburan,"[artis, kia, poetri, tengah, menjadi, sorotan,...","[artis, kia, poetri, tengah, jadi, sorot, publ...","[artis, kia, poetri, sorot, publik, nama, nya,..."
8,oktober lalu rencana pernikahan sojin girls da...,detik.com,hiburan,"[oktober, lalu, rencana, pernikahan, sojin, gi...","[oktober, lalu, rencana, nikah, sojin, girls, ...","[oktober, rencana, nikah, sojin, girls, day, a..."
9,penampilan lisa blackpink di kelab kabaret cra...,detik.com,hiburan,"[penampilan, lisa, blackpink, di, kelab, kabar...","[tampil, lisa, blackpink, di, kelab, kabaret, ...","[tampil, lisa, blackpink, kelab, kabaret, craz..."


Setelah menghilangkan stopwords menggunakan library nltk, data sudah memiliki kualitas yang baik untuk di proses ke representasi teks yang cocok untuk model. Selanjutnya, kita akan lanjut untuk melakukan representasi teks menggunakan metode countvectorizer.

In [11]:
# Clean dataframe to get only essential columns and perform proper column labelling
cols = ['Lemmatized_Text','media','label']
df = df[cols]
df = df.rename(columns={'Lemmatized_Text':'final_text'})

Sebelum itu, kita melakukan pelabelan kolom yang baik agar lebih mudah dalam pemrosesan.

In [12]:
df

,final_text,media,label
0,"[jakarta, manchester, united, hadap, suram, mu...",liputan6,olahraga
1,"[jakarta, pahlawan, dieluelukan, dukung, klub,...",liputan6,olahraga
2,"[luis, suarez, komentar, sukses, lionel, messi...",detik.com,olahraga
3,"[real, madrid, puas, hasil, redam, rayo, valle...",detik.com,olahraga
4,"[sepak, bola, profesional, anwar, el, ghazi, k...",media indonesia,olahraga
5,"[jakarta, grup, lawak, legendaris, indonesia, ...",liputan6,hiburan
6,"[jakarta, salah, film, bollywood, genre, komed...",liputan6,hiburan
7,"[artis, kia, poetri, sorot, publik, nama, nya,...",detik.com,hiburan
8,"[oktober, rencana, nikah, sojin, girls, day, a...",detik.com,hiburan
9,"[tampil, lisa, blackpink, kelab, kabaret, craz...",detik.com,hiburan


# First models using CountVectorizer method to represent the text

# Text Representation (CountVectorizer)

- Tujuan proses awal ini adalah menghasilkan suatu
set vektor dari data teks yang sudah dibersihkan dari simbol, hashtag, kata-kata “https”,
ataupun “.com”, dimana keseluruhan teks harus lower case dengan size vektor adalah 50, dan
frequency kata yang diperhitungkan minimal 3

In [13]:
# Perform CountVectorizer

# Menggunakan Lemmatized_Text sebagai input
text_data = df['final_text']

def custom_tokenizer(tokens):
    return tokens

# Inisialisasi objek CountVectorizer dengan membatasi jumlah fitur (panjang vektor) menjadi 50
# dan memperhitungkan kata-kata yang muncul minimal 3 kali
count_vectorizer = CountVectorizer(max_features=50, min_df=3, lowercase=False, tokenizer=custom_tokenizer)

# Mengonversi teks menjadi matriks dokumen-term (vektor)
X = count_vectorizer.fit_transform(text_data)

# Dapatkan nama fitur (kata) menggunakan metode get_feature_names()
feature_names = count_vectorizer.get_feature_names()

# Print the transformed matrix
print("Transformed Matrix:")
print(X.toarray())

# # Print words corresponding to each column
# for i, word in enumerate(feature_names):
#     column_values = X[:, i].toarray().flatten()
#     print(f"Word: {word}, Occurrences: {column_values}")


Transformed Matrix:
[[ 0  2  0  0  0  0  0  1  0  0  0  0  1  1  1  1  0  0  1  0  1  1  4  0
   1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  0  2  1  1  0
   1  1]
 [ 0  1  1  0  0  0  0  1  0  0  0  0  0  1  1  4  0  0  1  1  1  1  2  0
   1  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  1  0  1  1  1  0
   1  1]
 [ 0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  2  1  2  0  0  0  0  5  1
   0  0  0  0  0  1  0  0  0  0  0  4  0  0  0  0  0  0  0  1  0  0  0  0
   0  0]
 [ 1  0  0  0  0  1  0  0  0  0  0  0  2  0  0  2  0  0  0  2  0  0  1  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4  0  0  0
   0  0]
 [ 3  0  3  0  0  0  1  0  0  1  2  0  2  0  0  4  1  0  0  1  0  0 10  4
   0  1  0  0  0  0  0  0  0  0  0  0  0  2  0  2  1  3  0  1  1  0  0  1
   0  0]
 [ 0  1  0  6  0  1  0  3  0  1  0  0  0  1  1  0  0  0  1  0  1  2  1  0
   1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  0  1  1  1  1
   3  1]
 [ 0  1  0  8  0  2  0  1  0  0  0  0 

Disini data diubah ke matrix menggunakan CountVectorizer dengan jumlah matriks sebesar 50 dan jumlah minimum kemunculan kata adalah 3 untuk dapat diinklusikan ke dalam vektor. Selanjutnya, kita akan melakukan label encoding dan data splitting agar format data cocok untuk diproses oleh model.

In [14]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# for i in range(len(df)):
#     df["Lemmatized_Text"][i] = ' '.join(df["Lemmatized_Text"][i])
# # for i in df['Lemmatized_Text']:
# #     joined_text = ' '.join(i)
    
# tfidf_vectorizer = TfidfVectorizer(lowercase=False,max_features=50, min_df=3)

# tfidf = tfidf_vectorizer.fit_transform(df["Lemmatized_Text"])

# Label Encoding y

In [15]:
le = LabelEncoder()
df['label'] =  le.fit_transform(df['label'])

Disini, pelabelan data y dilakukan menggunakan library label encoder untuk mengubah data ke numerik yang cocok untuk model SVC dan RFC. label 0 merepresentasikan kategori hiburan, 1 merepresentasikan kategori olahraga, dan 2 merepresentasikan kategori politik.

# Data splitting (80% Train 20% Test)

In [16]:
# Assume you have a target variable 'labels' indicating the categories (politics, entertainment, sport)
labels = df['label']

test_indices = [1,7,11]  # Indices of rows to be included in the test set

X_test = X[test_indices]
y_test = labels[test_indices]

# Indices of rows not included in the test set will be used for training
train_indices = [0,2,3,4,5,6,8,9,10,14,12,13]

X_train = X[train_indices]
y_train = labels.iloc[train_indices]

Disini, data di split menjadi 20 persen test data dan 80 sisanya untuk training data.

# Modelling

In [17]:
# SVM Classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)

# Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)

# Print accuracy and classification report for both classifiers
print(f"SVM Accuracy: {svm_accuracy:.2f}")
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")

print("\nClassification Report for SVM:")
print(classification_report(y_test, svm_predictions))

print("\nClassification Report for Random Forest:")
print(classification_report(y_test, rf_predictions))


SVM Accuracy: 1.00
Random Forest Accuracy: 1.00

Classification Report for SVM:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



Dari classification report dan accuracy score, kedua model yang menggunakan metode representasi teks countvectorizer mendapatkan hasil akurasi 100 persen. Hal ini dapat disebabkan oleh fungsi dari metode countvectorizer dimana countvectorizer menghasilkan vektor matriks frekuensi dari setiap kata dimana hasil dari frekuensi tersebut memudahkan model untuk mengklasifikasikan data sesuai label dan mencapai hasil yang akurat. Countvectorizer juga memudahkan model untuk mencapai hasil yang akurat meskipun dataset jumlahnya sedikit, maka dari itu hasil kedua model diatas mendapat hasil yang baik. namun perlu diketahui, data yang tersedia jumlahnya sedikit sehingga hal ini dapat menyebabkan overfitting pada model. langkah selanjutnya yang dapat dilakukan setelah projek ini selesai adalah mungkin dengan menambah jumlah data pada dataset dengan melakukan scraping pada lebih banyak sumber.

# Second models using Word2Vec method to represent the text

# Text Representation (Word2Vec Embedding)

In [18]:
import numpy as np
from gensim.models import Word2Vec

# Tokenize the lemmatized text data
tokenized_text = [text for text in df['final_text']]

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_text, vector_size=50, window=5, min_count=3, sg=0)

# Function to convert text data to Word2Vec embeddings
def word2vec_embeddings(texts, model, max_length=50):
    embeddings_matrix = []
    for text in texts:
        embeddings = []
        for word in text:
            if word in model.wv:
                embeddings.extend(model.wv[word])
        # Pad the embeddings if the number of words in the text is less than max_length
        while len(embeddings) < max_length * model.vector_size:
            embeddings.extend([0] * model.vector_size)
        embeddings_matrix.append(embeddings[:max_length * model.vector_size])
    return np.array(embeddings_matrix)

# Convert the lemmatized text data to Word2Vec embeddings
word2vec_X = word2vec_embeddings(tokenized_text, word2vec_model)

# Print the flattened Word2Vec embeddings for the first example
print("Flattened Word2Vec Embeddings for the First Example:")
print(word2vec_X[0])

Flattened Word2Vec Embeddings for the First Example:
[ 0.02037464  0.00157795 -0.00999265 ... -0.01838439 -0.00470976
  0.00557227]


Selanjutnya, kita lanjut dengan melakukan modelling menggunakan model yang sama namun dengan menggunakan metode representasi teks yang berbeda yakni metode Word2Vec embeddings. Dimana representasi word2vec ini akan menghasilkan matriks yang sudah dikonversi menjadi shape 1d atau flattened dengan ukuran 50 dan jumlah minimum kemunculan kata agar dimasukkan ke word2vec vector adalah sebanyak 3 kali. Selanjutnya, kita akan membagi data menjadi 80% train data, dan 20% test data.

# Data splitting (80% Train 20% Test)

In [19]:
labels = df['label']

test_indices = [1,7,11]  # Indices of rows to be included in the test set

X_test2 = word2vec_X[test_indices]
y_test2 = labels[test_indices]

# Indices of rows not included in the test set will be used for training
train_indices = [0,2,3,4,5,6,8,9,10,12,13,14]

X_train2 = word2vec_X[train_indices]
y_train2 = labels.iloc[train_indices]

Setelah mempartisi data word2vec matriks 80% ke training set, dan 20% ke test set, kita dapat lanjut ke tahap modelling untuk mendapatkan akurasi dari data word2vec embedded ini.

# Modelling

In [20]:
# SVM Classifier
svm_classifier = SVC()
svm_classifier.fit(X_train2, y_train2)
svm_predictions = svm_classifier.predict(X_test2)
svm_accuracy = accuracy_score(y_test2, svm_predictions)

# Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train2, y_train2)
rf_predictions = rf_classifier.predict(X_test2)
rf_accuracy = accuracy_score(y_test2, rf_predictions)

# Print accuracy and classification report for both classifiers
print(f"SVM Accuracy: {svm_accuracy:.2f}")
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")

print("\nClassification Report for SVM:")
print(classification_report(y_test, svm_predictions))

print("\nClassification Report for Random Forest:")
print(classification_report(y_test, rf_predictions))

SVM Accuracy: 0.33
Random Forest Accuracy: 0.67

Classification Report for SVM:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.11      0.33      0.17         3
weighted avg       0.11      0.33      0.17         3


Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1
           2       1.00      1.00      1.00         1

    accuracy                           0.67         3
   macro avg       0.50      0.67      0.56         3
weighted avg       0.50      0.67      0.56         3



Dapat dilihat pada skor akurasi dan hasil classification report bahwa hasil akurasi yang didapatkan oleh model SVC adalah 33% sedangkan model RFC adalah 67 persen, hal ini dapat disebabkan algoritma random forest classifier yang menggunakan teknik averaging dari banyak pohon sehingga lebih tahan terhadap overfitting dibandingkan dengan algoritma SVC. Tingkat akurasi ini cukup jauh dibawah hasil akurasi model menggunakan representasi teks countvectorizer, hal ini dapat disebabkan oleh kurang efektifnya metode word2vec dalam dataset kecil. Metode word2vec merepresentasikan kata sebagai vektor, dimana kosakata yang memiliki kemiripan makna direpresentasikan lebih dekat di matriks vektor. Hasil representasi semantik ini memungkinkan model untuk menangkap dan mempelajari makna dan hubungan antarkata dengan lebih akurat. Dalam kasus ini, word2vec yang memerlukan jumlah data teks yang besar untuk dapat menangkap atau mendapatkan hubungan semantik antara kata dengan lebih baik, sehingga ketersediaan data yang sedikit, tentu mengurangi keefektifan dari model yang menggunakan metode word2vec tersebut, alhasil model yang kurang akurat daripada model yang menggunakan metode countvectorizer.